<a href="https://colab.research.google.com/github/allfed/CropOpt/blob/standardize/notebooks/yield_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# These Instructions Are Only Needed The First Time You Setup CoLab

In [ ]:
## Now we can clone the repo using SSH
!rm -rf allfed-integrated-model/
!git clone -b main git@github.com:allfed/allfed-integrated-model.git

In [ ]:
# This adds the Python files in our repo to our path so we can import
# them

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path) 

In [ ]:
# Now we'll install other dependencies from our Poetry file. We do this by first
# installing the toml package so we can read the pyproject.toml file
!pip install toml

In [ ]:
# Finally, we install the packages

import subprocess
import sys

def install_pip_package(package, version):
    subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package}=={version}"],"-U")

for package, version in pip_packages.items():
    if package == "python":
        continue
    install_pip_package(package, version.replace("^",""))

In [ ]:
# We then load the file and read our required packages from it
import toml
config = toml.load("allfed-integrated-model/pyproject.toml")
pip_packages = config["tool"]["poetry"]["dependencies"]

# Optimizer calculation demo

The purpose of this file is to allow researchers to run the optimizer with differing assumptions. Please reach out to morgan@allfed.info if you have questions about this code.

First, let's import the Optimizer, which runs through the code to optimize the minimum people fed of calories, fat, and protein. 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt

/home/dmrivers/Code/allfed-integrated-model


Now, we run the no resilient food case, generating the total food production with resilient foods.

In [2]:
print(src)

NameError: name 'src' is not defined

In [3]:
import src.run_model

/home/dmrivers/.local/lib/python3.9/site-packages/pulp/apis/coin_api.py:89: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")



no waste estimated people fed
13.595705

Food available after waste, feed ramp down and biofuel ramp down, with resilient foods
11.3642

Calculating 2100 calorie diet, excess feed to animals
Diet computation complete


Run the no resilient food case

In [8]:
import src.run_model_no_resilient_foods

/home/dmrivers/.local/lib/python3.9/site-packages/pulp/apis/coin_api.py:89: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")



Estimated people fed, no resilient foods, no waste
7.2441166

Estimated people fed, no resilient foods, minus waste & delayed halt of nonhuman consumption 
5.9240094



Run the baseline scenario

In [5]:
import src.run_model_before_catastrophe

KeyError: 'INDUSTRIAL_FOODS_SLOPE_MULTIPLIER'

Run the Monte Carlo scenarios

In [8]:
import src.monte_carlo

/home/dmrivers/.local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


i
0
optimization successful
{'fat': 13.01, 'kcals': 12.75, 'protein': 13.75}
Nutrients with constraining values are: ['kcals']
Estimated people fed is 12.746164 billion
i
1


/home/dmrivers/.local/lib/python3.9/site-packages/pulp/apis/coin_api.py:89: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")


optimization successful
{'fat': 11.25, 'kcals': 11.25, 'protein': 11.25}
Nutrients with constraining values are: ['fat', 'kcals', 'protein']
Estimated people fed is 11.25352 billion
i
2
optimization successful
{'fat': 11.84, 'kcals': 9.79, 'protein': 9.79}
Nutrients with constraining values are: ['kcals', 'protein']
Estimated people fed is 9.79182 billion
i
3
optimization successful

Scenario is impossible

i
4
optimization successful
{'fat': 11.87, 'kcals': 11.87, 'protein': 11.92}
Nutrients with constraining values are: ['fat', 'kcals']
Estimated people fed is 11.874818 billion
i
5


/home/dmrivers/Code/allfed-integrated-model/src/analysis.py:694: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
/home/dmrivers/Code/allfed-integrated-model/src/analysis.py:710: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
/home/dmrivers/Code/allfed-integrated-model/src/analysis.py:719: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


...

optimization successful
{'fat': 7.97, 'kcals': 7.97, 'protein': 9.2}
Nutrients with constraining values are: ['fat', 'kcals']
Estimated people fed is 7.9661883 billion
i
882
optimization successful
{'fat': 12.19, 'kcals': 12.19, 'protein': 12.21}
Nutrients with constraining values are: ['fat', 'kcals']
Estimated people fed is 12.193028 billion
i
883
optimization successful
{'fat': 13.45, 'kcals': 13.45, 'protein': 13.45}
Nutrients with constraining values are: ['fat', 'kcals', 'protein']
Estimated people fed is 13.450112 billion
i
884
optimization successful
{'fat': 12.27, 'kcals': 12.27, 'protein': 12.28}
Nutrients with constraining values are: ['fat', 'kcals']
Estimated people fed is 12.272661 billion
i
885
optimization successful
{'fat': 11.1, 'kcals': 11.1, 'protein': 11.21}
Nutrients with constraining values are: ['fat', 'kcals']
Estimated people fed is 11.100251 billion
i
886
optimization successful

Scenario is impossible

i
887
optimization successful
{'fat': 10.42, 'kcals': 10

optimization successful
{'fat': 10.02, 'kcals': 9.55, 'protein': 9.55}
Nutrients with constraining values are: ['kcals', 'protein']
Estimated people fed is 9.5496902 billion
i
926
optimization successful
{'fat': 10.45, 'kcals': 9.88, 'protein': 9.88}
Nutrients with constraining values are: ['kcals', 'protein']
Estimated people fed is 9.8797236 billion
i
927
optimization successful
{'fat': 12.12, 'kcals': 12.12, 'protein': 12.12}
Nutrients with constraining values are: ['fat', 'kcals', 'protein']
Estimated people fed is 12.124393 billion
i
928
optimization successful
{'fat': 11.03, 'kcals': 11.03, 'protein': 11.65}
Nutrients with constraining values are: ['fat', 'kcals']
Estimated people fed is 11.029792 billion
i
929
optimization successful
{'fat': 13.83, 'kcals': 13.83, 'protein': 13.83}
Nutrients with constraining values are: ['fat', 'kcals', 'protein']
Estimated people fed is 13.829812 billion
i
930
optimization successful
{'fat': 10.93, 'kcals': 9.57, 'protein': 9.57}
Nutrients wit

optimization successful
{'fat': 10.46, 'kcals': 9.66, 'protein': 9.66}
Nutrients with constraining values are: ['kcals', 'protein']
Estimated people fed is 9.6643863 billion
i
969
optimization successful
{'fat': 15.09, 'kcals': 15.09, 'protein': 15.09}
Nutrients with constraining values are: ['fat', 'kcals', 'protein']
Estimated people fed is 15.089576 billion
i
970
optimization successful
{'fat': 10.39, 'kcals': 10.39, 'protein': 10.39}
Nutrients with constraining values are: ['fat', 'kcals', 'protein']
Estimated people fed is 10.390426 billion
i
971
optimization successful
{'fat': 10.44, 'kcals': 10.44, 'protein': 10.44}
Nutrients with constraining values are: ['fat', 'kcals', 'protein']
Estimated people fed is 10.435455 billion
i
972
optimization successful
{'fat': 10.89, 'kcals': 10.89, 'protein': 11.02}
Nutrients with constraining values are: ['fat', 'kcals']
Estimated people fed is 10.891319 billion
i
973
optimization successful
{'fat': 10.15, 'kcals': 9.53, 'protein': 9.53}
Nutr

/home/dmrivers/.local/lib/python3.9/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


95% lower
2348.1980442307695
95% upper
4119.229120192308
number failed to optimize
84
